# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium.webdriver.chrome import service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/%E7%9C%8B%E7%9B%A4%E5%AE%A4/"  #財經看盤室新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
# using opera driver
webDriverServer = service.Service('./Data/operadriver_win64/operadriver.exe')
webDriverServer.start()

capabilities = {
    'operaOptions': {
        'binary': 'C:/Program Files (x86)/Opera/launcher.exe'
    }
}

browser = webdriver.Remote(webDriverServer.service_url, webdriver.DesiredCapabilities.OPERA)

browser.get(ETtoday_url)

# --- scroll to bottom ---
# get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

# load infinite page
while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # pause to load page
    time.sleep(1)
    
    # compare new scroll height with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    
# --- get page data ---
html_source = browser.page_source
soup = BeautifulSoup(html_source, 'html5lib')

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [25]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    #news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    #print(news_content)
    for p in news_content:
        print(p.get_text())

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [26]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 0050才第二！股民「人氣ETF排名」揭曉　最夯那檔入手沒


▲ETF人氣排行榜，可點圖放大。（圖／柴鼠兄弟授權提供）
記者林妤柔／綜合報導
近年來指數股票型基金（ETF）正夯，成為存股族心目中的選項之一。ETF到底該怎麼挑，才不會踩雷？對此，知名財經Youtuber柴鼠兄弟就分享7月份股民中人氣最高的ETF排行榜，想入手ETF的投資人不妨當作指標參考一下吧！
人氣最高的ETF前三名，分別是元大高股息（0056）、元大台灣50（0050）、國泰永續高股息（00878）；其中，00878是這個月的最大亮點。柴鼠兄弟表示，從7/20掛牌到7月底，短短幾天交易日，00878資產規模就一舉突破145億，受益人數直逼10萬大關，擠下元大臺灣ESG永續（00850）和富邦公司治理（00692），空降人氣排行榜第三名。
0056的2019年殖利率約6.7%，0050的2019年殖利率為4.2%，都是討論度相當高的ETF。柴鼠兄弟指出，在台股創歷史最高之際，反而幾檔明星ETF的人氣，在7月份都出現7-9%的下滑，「果然季配息又低門檻的威力還是不容小覷」。不過，就整體台股原型ETF的受益人數來說，7月還是成長7.5%，增加超過5萬人，總計來到73萬人，合計總資產規模也來到2287億。
另外，在人氣排行前十名當中，受益人數月增幅高達26%的是富邦科技（0052）。因為它目前有高達71%的權重是台積電（2330），所以在前一波台積電漲停的熱潮當中，成為最直接受惠的原型ETF，在同一天亮燈漲停，格外引起注目。
----------------------------------------------------------------------

[1] 郭明錤：玉晶光鏡頭質量卡關　iPhone 12鏡頭初期優先下單大立光


▲傳供應商玉晶光出了質量問題，iPhone 12可能轉單大立光。（渲染圖／PhoneArena）

記者謝仁傑／綜合報導
據天風國際分析師郭明錤的最新研究報告指出，iPhone 12 的相機鏡頭供應商之一有質量問題，但若蘋果的其他供應商可以彌補這些問題，就不太會對蘋果的發貨時間表產生影響。
郭明錤指出，玉晶光（


▲台股終場收在12828點。（資料照／記者湯興漢攝）
記者陳心怡／台北報導
美股電子盤跌幅擴大，台股今 (7) 日在蘋概三王等大型電子權值股熄火，加上櫃買指數漲翻跌，導致指數開低震盪走低，盤中一度跌破 12800 點，最低跌至12791點，終場收在12828點，下跌84點或0.66%，成交量2281.07億元。
蘋概三王中，台積電（2330）以 433元作收、下跌0.46%；鴻海（2317）以78.3元作收、下跌0.89%；股王大立光以4035元作收、下跌3%。
其他大型權值股除了台塑、台化上漲，中華電收平盤外，其餘都下跌。
今日上市類股漲跌互見，其中，電器類股表現最好，上漲1.42%，表現較差為光電類股，下跌2.12%。
此外，康友KY 子公司六安華源製藥，傳出公司機器設備被贛州銀行廈門分行抵押，公司澄清未有此事，但仍遭證交所列為全額交割股，今日股價開盤跌停鎖死，委賣張數逾 2 萬張。
兆豐投顧協理黃國偉表示，台股昨天挑戰上禮拜高點沒過，今日逢上禮拜壓力，再加上亞股普遍表現低迷，讓台股也面臨技術性壓力，也產生利多反應鈍化。
展望後市，黃國偉指出目前台股普遍不便宜，股價幾乎都太高，建議投資人找本益比低的個股，不要再追題材相關類股。
----------------------------------------------------------------------

[12] 美國銀行上調蘋果目標價至470美元　距離市價還有3%上漲空間！


▲美國銀行將蘋果目標價從420美元上調至470美元。（圖／達志影像／美聯社）
記者林鼎閎／綜合報導
分析當前的估值與市場行情後，美國銀行將蘋果的投資建議，從「買入」下調至「中性」，但卻將目標價從420美元上調至470美元。蘋果昨天股價上漲3.49％，來到455.61美元，距離470美元還有約3％上漲空間。
美國銀行分析師Wamsi Mohan表示「蘋果股價自6月份以來漲勢兇猛，而市場對該公司2021年的估值卻保持不變。」
Wamsi Mohan認爲「5G iPhone存在風險，如果平均銷售價格過高，可能會給產品毛利率和銷量帶來壓力。服務利潤可能會受到內容攤銷成本的擠壓。另外，蘋果在2021年也將面臨艱難的競爭。」
Mohan設定新的蘋果目標價為470美元，他認為「蘋果每股獲利17美元中，有6美元將來自服務業務，其餘11


▲台翰越南廠。（圖／台翰提供）
記者姚惠茹／台北報導
台翰精密（1336）今（6）日公布7月合併營收1.78億元，月增19.17%，創今年來單月新高，主要是受到疫情趨緩，東南亞各地區政府以漸進式復甦經濟活動，累計今年前7月合併營收9.9億元，年減26.22%。台翰表示，最壞的情況已過，下半年業績可望比上半年好。
台翰表示，上半年營運受到新冠肺炎疫情影響，但菲律賓廠區自5月16日復工後，可見主要客戶的拉貨力道有逐步恢復的跡象，公司與客戶積極拉高產能稼動率以因應整體印表機/事務機的需求，目前產能稼動率已達50%水準，而越南廠的產能利用率也已達70%。
台翰指出，雖然疫情短期影響訂單遞延出貨，但目前已與主要客戶積極針對新機種進行認證，若順利按照時程進度，有機會在今年第3季試量產，未來隨著客戶需求增加，將有助於整體產能規模放大，並強調最壞的情況已過，下半年本業營運有望恢復過往成長表現，可望比上半年好。
台翰說明，目前越南廠區佔整體營收55%、菲律賓廠區約佔35%、東莞廠約佔10%，而且東莞廠將會持續縮編，預計明年僅剩60人左右，作為模具開發設計使用，主要是因為大陸生產成本越來越高，但今年由於有新模具開發，所以第3季的比重則是以東莞廠為主。
根據中國海關總署公布今年上半年最新貿易統計數據顯示，大陸與所有國家及地區的貿易額表現來看，由於大陸企業將生產線轉移至東盟國家，創造大陸與東盟(ASEAN)的進出口貿易額首次升至第一，而且近期歐美、日韓等各國也積極祭出補貼計畫，鼓勵該國企業遷回該國家或東南亞區藉以分散產能布局、確保產品與材料的順利供應，有利於東南亞產業鏈群聚發展。
展望下半年營運，台翰表示，看好東南亞兩個生產基地產能規模放大，貢獻整體營運穩健基礎，雖然東南亞地區政府仍實施防疫措施，但仍以最低限度生產方式發揮最大量產價值，滿足主要客戶的拉貨需求，有助於下半年營運保持良好的成長動能。
台翰強調，公司持續強化模具設計、塑膠射出的量產技術，將憑藉著深耕東南亞市場超過15年，積極爭取更多與不同產業客戶合作的可能性，藉以保持高度的訂單能見度，並對整體營運帶來正面的挹注。
此外，台翰閒置資產處分利益2.13億元已在今年第2季入帳，同時也在上半年完成庫藏股買回4000張，目前股本為7.71億元，未來台翰仍將持續落實充實營運資金、強化財務結構、提高財務運用靈活度及提升股東權益，致力將不確定


▲Apple TV。（圖／取自THE VERGE）

記者謝仁傑／綜合報導
根據最新數據顯示，串流媒體設備和電視機上盒（像是 Roku 或 Apple TV），面臨越來越多與互聯網的智慧型電視挑戰，且價格持續下降，許多消費者都選擇了這項多合一的方案。
根據《The Information》的數據顯示，到2020年初，美國擁有智慧型電視的家庭比例達到54％，較去年同期增長了47％，超過了所有串流媒體設備和機上盒所佔的42％。市場之間的差距可能會繼續擴大，美國消費者技術協會估計，到2020年消費者將購買3500萬台智慧型電視，而串流媒體設備則為2200萬台。
儘管有大約一半的智慧型電視用戶也會使用串流媒體設備，但新一代的智慧型電視可以跳過機上盒或串流媒體設備，仍具備其功能。目前在串流媒體設備中，以 Roku 的38％市占率最高，緊追在後的是 Fire TV（佔32％）和 Apple TV（佔13％），據傳一些娛樂公司正在討論與智慧型電視製造商，討論繞過機上盒直接合作的方式。
儘管智慧型電視的日益普及可能會影響 Roku 和 Apple TV 等串流媒體設備的銷售，但蘋果服務部門已準備好利用這一點，允許用戶在許多第三方智慧型電視和串流媒體設備上，觀看 Apple TV+ 和使用其他 Apple TV 的應用程式。
蘋果可望在今年推出新的 Apple TV 版本，據傳會提高內存和性能，且似乎也正在整合遊戲和家庭自動化功能。
＃ 對面的女孩看過來～～
----------------------------------------------------------------------

[34] 台塑四寶7月營收914.51億元　庫存回補需求升溫


▲圖左至右分別為：台塑董事長林健男、南亞董事長吳嘉昭、台化副董洪福源、台塑化總經理曹明。（資料照／記者余弦妙攝）

記者余弦妙／台北報導
台塑四寶今（6）日公布今年7月獲利，四大公司台塑、台塑化、台化、南亞等四大公司營收與6月相比成長4.5%，7月合計營收為914.51億元，為連續第二個月反彈，復甦態勢明顯。
台塑7月合併營業額157億1,628萬元，較上月成長6.4％，銷售量、價都成長。台塑表示，印度受疫情持續惡化及封城影響，PVC管材加工廠被迫停工，不過，7月為PE傳統旺季，市場需求增加，加上中國大陸石化廠集中歲修，


▲美元換匯價持續創低。（圖／記者湯興漢攝）
記者陳心怡／台北報導
 有換美元需求的民眾注意了！新台幣今 (6) 日開盤以29.480元兌1美元，升值2.6分開出，開盤不久後隨即站上29.330元價位，並持續向上走勢，一度觸及29.305元，大漲2.01角，這也讓美元換匯價再創新低。根據台銀今天上午牌告的外幣現金賣出匯價，美元匯價來到29.585元，相較於2月底的今年最高點匯價，換新台幣10萬如今可以多拿到124美元，等於多賺了新台幣約3670元。
由國際資金與法人圈參與的台北外匯市場，新台幣兌美元匯率近期持續走強，今日以1美元兌換 29.48元新台幣、升值2.6分開出後，升值幅度持續擴大，盤中最高來到 1：29.305。
一般民眾若想要赴銀行換美元，則要看銀行牌告的現金賣出匯價。以台銀來說，今天的美元現金賣出匯價來到29.585元，是2018年4月19日（匯價29.532元）、2年多以來新低價。對於有換美元需求的民眾來說，無疑是一大福音。
若想換新台幣10萬元，今天可以換到3380美元；相對今年高點價位（2月24日）的匯價30.71元來計算，當時只能換到3256美元，相差了124美元，如今等於多賺到新台幣約3670元。
----------------------------------------------------------------------

[44] 快來一起戳破它！　蘋果製作彩蛋慶祝Apple Store 10周年


▲蘋果製作彩蛋慶祝Apple Store App 10周年。（圖／取自MacRumors）

記者謝仁傑／綜合報導
自Apple Store應用程式推出以來已有10年了，蘋果為了慶祝這個里程碑，在Apple Store中搜尋「10年（10 years）」，就會跳出一個隱藏的特效彩蛋。
蘋果用戶可以直接在Apple Store這個App上購買Apple產品，至今已營運了10年。今日，在Apple Store搜尋列鍵入「10 years」，就會有兩顆圖樣標示「10」的藍色氣球從螢幕底部向上升起，氣球會在螢幕框內反彈一會兒，然後飄走，如果用戶觸擊它們，則可以把它們彈破，相當有趣。
蘋果至今打破了許多里程碑，目前為市值最高的企業，市值已經逼近2兆美元（約新台幣59兆元）。
pic.twitter.com/SKKmLSrm7O
＃ 對面


▲統振。（圖／統振提供）
記者姚惠茹／台北報導
統振（6170）今（5）日召開董事會通過重大決議，宣布出售旗下子公司統達能源29％股權予達方電子，貢獻每股稅後盈餘0.4元，股權出售後仍保有統達33.65％股權，未來仍可持續享有轉投資收益，並宣告正式轉型為高毛利的利基型金融服務業。
統振表示，鋰電池研發、生產與銷售為統振主營業務之一，但因產業競爭激烈，2018年12月統振取得金管會金融監理沙河實驗許可，正式跨足外籍移工跨境匯款業務後，營運表現開始明顯成長，再加上佈局電動自行車，帶動整體鋰電池業務轉虧為盈，推升2019年合併營收38.5億元，創5年來新高，稅後淨利0.86億元、每股稅後盈餘（EPS）0.96元，更創10年來的新高表現。
統振指出，這次出售旗下子公司統達29％股權予達方電子，一方面基於與達方策略聯盟可望產生極大綜效，開展鋰電池業務外，另一方面，更確立公司未來核心發展主軸，將轉型為高毛利的利基型金融服務業者，並以現有的外籍移工跨境匯款業務為基礎，鎖定國內70萬名外籍移工，發展電子商務、分期金融等其他業務，目前轉型成效已經顯現。
統振說明，2019年5月正式跨足外籍移工跨境匯款業務，並在高毛利業務的帶動下，去年獲利創10年新高，今年第1季雖因鋰電池業務受到新冠肺炎疫情影響，致合併營收相較去年同期減少7％，但在高毛利的移工匯款業務持續高成長下，毛利率達20％，年增6個百分點，並帶動稅後淨利0.26億元，年增152.28％，每股稅後盈餘0.3元。
統振補充，今年以來移工匯款業務每月仍維持高度成長，在相關費用固定下，經濟規模的擴大預期可推升毛利率再攀升，而且相較國內約71萬名外籍移工，每月總匯款筆數達100萬筆來看，統振目前在移工跨境匯款市場的占有率仍低於10％，並由於公司所提供為每筆匯款只收取固定手續費，相較銀行匯款更低，所以在政府許可的合法業務等多重誘因下，預期未來仍具有相當的成長空間。
統振強調，董事會這次通過出售統達29％股權，預計將可貢獻每股稅後盈餘0.4元，並將會在第3季認列，而統振旗下另一鋰電池子公司達振能源，主力市場在3C商品，去年已轉虧為盈，今後將朝向綠色能源相關產品全力發展。
----------------------------------------------------------------------

[56] 慧洋海運7月轉虧


▲大同董事長林郭文艷。（圖／記者陳心怡攝）
記者陳心怡／台北報導
大同董事長林郭文艷昨（4）日申報轉讓手中大同股票6,300張，相當於約45%持股都將出脫，以大同昨日收盤價每股14.5元估計，約可套現9135萬元。今日大同公司也證實，林郭文艷申報轉讓持股6300張為質設股票。
林郭文艷昨日申報轉讓持股6300張，採一般交易方式進行轉讓，約占她個人持股1萬4004張173股的44.98%，如果成功轉讓，持股將剩7704張173股。此外，林郭文艷持股當中已質設9931張。
法人表示，林郭文艷申報轉讓持股6300張因占持股的44.98%，若全數賣出之後，再成功申讓持股達選任董事時1/2，董事資格將自動解任。
大同公司也證實林郭文艷這次賣股是將原本質設的股票賣出。市場解讀，大同股價近期不斷暴跌，造成質設股票維持率大降，又無法補足保證金，遭銀行等金融單位強迫斷頭賣出。
外資近來也擴大賣超大同股票，昨日持續賣超大同9933張，連12賣，大同股價今日以14.2元開低，一度下探至13.95元，下跌3.79%。大同近期股價重挫，股價從21.6元附近拉回，今日下探至波段低點13.95元，波段跌幅近36%。
----------------------------------------------------------------------

[67] 永續投資ETF秒懂！　喜歡貼近大盤的選元大、愛中小型股選富邦、想領股利挑國泰


▲ESG已成為投資新選擇。（圖／《ETtoday》資料照）

記者陳心怡／台北報導
ESG（環境、社會、及公司治理）的永續投資概念，已然成為國際趨勢，尤其在疫情爆發後，全球社會經濟環境整個大改變，企業的ESG表現，皆面臨更嚴格的檢視。台灣自從去年元大投信發行元大台灣ESG永續ETF基金（00850）後，國泰投信也在上個月發行國泰永續高股息ETF基金（00878），再加上另外一檔富邦公司治理ETF基金（00692），光這3檔ETF規模就超過200億元。
ESG分別代表環境(Environmental)、社會(Social)、及公司治理(Governance)，換句話說ESG是衡量企業社會責任(Corporate Social Responsibility, CSR)的重要指標。
ESG企業社會責任是指企業在創造利潤、對股東利益負責的同時，還要承擔對所有


▲ESG企業債指數的平均信用評級為投資等級，信用風險程度相對較低。（圖／pixabay）
記者楊絡懸／台北報導
新冠肺炎（COVID-19）疫情及地緣政治風險持續干擾市場，但受惠於全球中央銀行持續降息及美國聯準會（Fed）大規模的救市措施，支撐ESG企業債指數7月以來維持走升格局。
柏瑞投信表示，在當前低利率、且全球負利率債券規模居高不下的背景下，資金將持續流入平均信用評等較佳、利差仍有收斂空間的ESG企業債，預期下半年還有看升空間。
柏瑞投信於7月下旬發布業界第一份「洞悉ESG」報告，獲得市場關注與回響；8月上旬再發布第二份「洞悉ESG」報告，進一步分析ESG企業債市情況。
根據柏瑞投信最新的「洞悉ESG」報告指出，在全球主要央行高度寬鬆的政策支持下，ESG債券的初級市場相當熱絡，ESG企業債指數成分企業的發債狀況持續回溫，且遭降評的家數明顯放緩。
截至5月底，ESG企業債指數的成分企業共有2319家，總計5月份有29家信用評級被調升、27家被調降；不過，ESG企業債指數的平均信用評級為投資等級，信用風險程度相對較低。

▲柏瑞投信也提醒，未來要持續觀察全球疫情是否再創高峰，以及美國總統大選和地緣政治關係的後續發展，並留意ESG企業債市仍有波動風險。（圖／pixabay）
柏瑞ESG量化債券基金經理人施宜君表示，在前景不明、市場波動劇烈之際，信用評級較好的ESG企業債指數不但表現抗跌，回復速度也很快。今年來ESG企業債指數已收復先前跌幅，到7月底漲幅已超過5%。
此外，就算3月份市場恐慌情緒最甚之時，資金僅小幅度流出全球ESG債券ETF，近3個月則轉為持續流入。
從評價面來看，6月份全球ESG債券與公債之利差收窄約25個基本點至155個基本點，此利差水準仍高於近5年均值，顯示ESG債券的評價具有吸引力。
全球ESG債券的殖利率6月份下滑約26個基本點至1.86%，在Fed將維持近0%利率的水準至2022年的環境下，這樣的殖利率仍相對具投資價值。
施宜君表示，在投組配置策略上，以成熟國家為主體，美國的比重最高，並相對偏好防禦型產業，以金融業占比最高。至於在信評配置上，以投資等級債為主。
不過，柏瑞投信也提醒，未來要持續觀察全球疫情是否再創高峰，以及美國總統大選和地緣政治關係的後續發展，並留意ESG企業債市仍有波動風險。
【延伸閱讀】
►柏瑞投信首份「洞悉ESG」報


 ▲50多年前，滑鼠剛誕生時長這樣。（圖／翻攝自國外網9to5mac.com）
記者吳佳穎／綜合報導
滑鼠的發明人英格利西（William English）過世，享耆壽91歲。他將滑鼠帶給這個世界，被譽為是蘋果電腦（Mac）及Windows作業系統的啟發者和先驅者。

▲滑鼠共同發明人英格利西過世。（圖／翻攝自維基百科）
據了解，早在蘋果1983年發表Lisa電腦，以及1984年發表麥金塔作業系統之前，全球第一支滑鼠已經在1968年問世於史丹福國際研究所 （Stanford Research Institute），由研究員恩格巴特（Douglas Englebart）和英格利西共同發明，恩格巴特已於2013年世。兩人被視為人機介面研究的先驅。
在1968年的滑鼠DEMO秀裡，影片配音稱恩格巴特為「誕生滑鼠的人」；不過根據恩格巴特留下的手稿筆記， 似乎英格利西才是滑鼠的「製造者」。
據說，當時是由恩格巴特先有滑鼠的想法概念，繪製素描草圖，再由英格利西實際製作出來。而在1968年的展示會上，恩格巴特大量露臉，展示這些概念，英格利西則在幕後輔助協作。
在影片45秒起處，可以看到當時的滑鼠，操作的情況。整個滑鼠看來很陽春，只有一個小木盒加上一些簡單的零件，不過，卻成為影響後世的重要創作。時至今日，滑鼠依舊是人機介面操作的主流，這場50年前發表的DEMO秀，也可以視為超聯結、螢幕共享、多人協作及視訊的始祖。許多電腦作業，甚至是DEMO模式都能從影片中找到原始型式。
不少人認為，蘋果（Mac）電腦的成功部分要歸功於滑鼠的發明，讓人可以透過「觸控」電腦，發揮工作最大潛能。
另外，蘋果迷指出，2020年向世界告別的「電腦先驅者」不只英格利西，還有「複製貼上」的共同發明人拉里·泰斯勒（Larry Tesler）也於今年2月過世。泰斯勒正是向賈伯斯（Steve Jobs）展示PARC圖形應用介面和滑鼠的工程師之一。
▼1968年的滑鼠DEMO秀。

----------------------------------------------------------------------

[88] 台股收復昨日失土！台積電帶頭漲逾2%　指數大漲196點收12709點


▲台股終場收在12709點。（資料照／記者湯興漢攝）
記者陳心怡／台北報導
在微軟、蘋果領軍下，美股昨日收高，其